# Algoritmos de Bag of Words

In [2]:
import math
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext

## BAG OF WORDS

In [3]:
def BAG_OF_WORDS(data):
    
    # Tokenizacion de la data, y conversion a lowercase
    tokens=data.flatMap(lambda x: [(i.lower(),len(i)) for i in x[1].split()])
    # Agrupamos las keys
    mapeo=tokens.groupByKey()
    bow=mapeo.map(lambda x: x[0])
    # Filtramos a aquellas keys que tengan un tamanio mayor a '2'
    bow = bow.filter(lambda x: len(x) > 2)
    
    return bow
    

In [4]:
# Creamos nuestra data de dos oraciones
data=[(0,"el es muy bueno en mineria de datos"),
      (1,"el sabe minerar datos con librerias de datos")]

# Volvemos este arreglo de tuplas en un RDD de 2 particiones
oraciones=sc.parallelize(data,2)

# BAG OF WORDS
bow = BAG_OF_WORDS(oraciones)
bow.collect()

['bueno', 'mineria', 'datos', 'con', 'muy', 'sabe', 'minerar', 'librerias']

# TF

In [5]:
def TF(oraciones):
    
    # Tokenizamos cada oracion, y le agregamos una nueva variable para el conteo de apariciones
    mapeo = oraciones.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])

    # Agruparemos los pares key-value comunes, y agregaremos los valores de la misma para oftener sus apariciones
    reducir = mapeo.reduceByKey(lambda x,y:x+y)

    # APlicaremos la formula de tf para cada token , en cada documento
    tf = reducir.map(lambda x: (x[0][1], (x[0][0],(1 + math.log10(x[1]/(len(data[x[0][0]-1][1].split(" "))))))))
    reducir2 = reducir.map(lambda x: ((x[0][0],x[0][1]), (1 + math.log10(x[1]/(len(data[x[0][0]][1].split(" "))))) ))

    # Visualizacion de los tokens con sus valores TF
    muestra = reducir.map(lambda x: (x[0][1], "doc-"+str(x[0][0]),(1 + math.log10(x[1]/(len(data[x[0][0]][1].split(" ")))))))
    
    return tf, reducir2, muestra

In [6]:
# Creamos nuestra data de dos oraciones
data=[(0,"el es muy bueno en mineria de datos"),
      (1,"el sabe minerar datos con librerias de datos")]

# Volvemos este arreglo de tuplas en un RDD de 2 particiones
oraciones=sc.parallelize(data,2)
oraciones.collect()

# TF
tf = TF(oraciones)
tf[2].collect()

# Visualizacion de TF en tabla
hasattr(tf[2], "toDF")
spark = SparkSession(sc)
hasattr(tf[2], "toDF")
tf[2].toDF(["Token","DocumentoID","TF"]).show()

+---------+-----------+-------------------+
|    Token|DocumentoID|                 TF|
+---------+-----------+-------------------+
|       el|      doc-0|0.09691001300805646|
|    bueno|      doc-0|0.09691001300805646|
|       en|      doc-0|0.09691001300805646|
|  mineria|      doc-0|0.09691001300805646|
|    datos|      doc-0|0.09691001300805646|
|     sabe|      doc-1|0.09691001300805646|
|  minerar|      doc-1|0.09691001300805646|
|librerias|      doc-1|0.09691001300805646|
|       de|      doc-1|0.09691001300805646|
|       es|      doc-0|0.09691001300805646|
|      muy|      doc-0|0.09691001300805646|
|       de|      doc-0|0.09691001300805646|
|       el|      doc-1|0.09691001300805646|
|    datos|      doc-1| 0.3979400086720376|
|      con|      doc-1|0.09691001300805646|
+---------+-----------+-------------------+



# TF-IDF

In [7]:
def TF_IDF(reducir2, tf):
    
    # Asignamos los pares key-value a uno nuevo, el key será el token, value será TF y un auxiliar(1)
    mapeo2=reducir2.map(lambda x: (x[0][1],(x[0][0],x[1],1)))

    # Extraemos el token y auxiliar, esto nos ayudará a identificar la ocurrencia en las oraciones
    mapeo3=mapeo2.map(lambda x:(x[0],x[1][2]))

    # Reducimos por key, para obtener el conteo de apariciones en el documento
    reducir3=mapeo3.reduceByKey(lambda x,y:x+y)

    # Calcularemos el valor de IDF
    idf=reducir3.map(lambda x: (x[0],math.log10(1 + len(data)/x[1])))

    # RightOuterJoin entre valores tf y idf para unir el TF y IDF de cada token
    join = tf.rightOuterJoin(idf)

    # Multiplicamos los valores de TF y IDF para obtener TF-IDF
    tfidf = join.map(lambda x: (x[0],"doc-"+str(x[1][0][0]), x[1][0][1]*x[1][1]))

    return tfidf

In [8]:
# Creamos nuestra data de dos oraciones
data=[(0,"el es muy bueno en mineria de datos"),
      (1,"el sabe minerar datos con librerias de datos")]

# Volvemos este arreglo de tuplas en un RDD de 2 particiones
oraciones=sc.parallelize(data,2)
oraciones.collect()

# TF-IDF
tf = TF(oraciones) # Aplicamos tf
tfidf = TF_IDF(tf[1],tf[0]) # Aplicamos tfidf con el tf anterior

# Visualizacion de TF-IDF en tabla
hasattr(tfidf, "toDF")
spark = SparkSession(sc)
hasattr(tfidf, "toDF")
tfidf.toDF(["Token","DocumentoID","TF-IDF"]).show()

+---------+-----------+-------------------+
|    Token|DocumentoID|             TF-IDF|
+---------+-----------+-------------------+
|       el|      doc-0| 0.0291728207956116|
|       el|      doc-1| 0.0291728207956116|
|    datos|      doc-0| 0.0291728207956116|
|    datos|      doc-1|0.11979187908506812|
|      con|      doc-1|0.04623782700130271|
|     sabe|      doc-1|0.04623782700130271|
|  minerar|      doc-1|0.04623782700130271|
|librerias|      doc-1|0.04623782700130271|
|       es|      doc-0|0.04623782700130271|
|    bueno|      doc-0|0.04623782700130271|
|       en|      doc-0|0.04623782700130271|
|  mineria|      doc-0|0.04623782700130271|
|       de|      doc-1| 0.0291728207956116|
|       de|      doc-0| 0.0291728207956116|
|      muy|      doc-0|0.04623782700130271|
+---------+-----------+-------------------+



# N-GRAMS

In [9]:
def BIGRAMS(data):
    # Tokenizamos y agrupamos en bigramas
    bigrams = [b for l in data.collect() for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
    
    return bigrams

In [10]:
# Creamos nuestra data de dos oraciones
data=["el es muy bueno en mineria de datos",
      "el sabe minerar datos con librerias de datos"]

# Volvemos este arreglo de tuplas en un RDD de 2 particiones
oraciones=sc.parallelize(data,2)
oraciones.collect()

# BI-GRAMS
bigrams = BIGRAMS(oraciones) # Aplicamos tf
bigrams

[('el', 'es'),
 ('es', 'muy'),
 ('muy', 'bueno'),
 ('bueno', 'en'),
 ('en', 'mineria'),
 ('mineria', 'de'),
 ('de', 'datos'),
 ('el', 'sabe'),
 ('sabe', 'minerar'),
 ('minerar', 'datos'),
 ('datos', 'con'),
 ('con', 'librerias'),
 ('librerias', 'de'),
 ('de', 'datos')]